# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 14 2022 12:08PM,250697,19,ADV80007664,"AdvaGen Pharma, Ltd",Released
1,Nov 14 2022 12:08PM,250697,19,ADV80007665,"AdvaGen Pharma, Ltd",Executing
2,Nov 14 2022 12:08PM,250697,19,ADV80007666,"AdvaGen Pharma, Ltd",Released
3,Nov 14 2022 12:03PM,250696,12,HH8335,Hush Hush,Released
4,Nov 14 2022 12:01PM,250695,20,8774786,"Andelyn Biosciences, Inc.",Released
5,Nov 14 2022 11:36AM,250685,10,MTR-22-0268,"Senseonics, Incorporated",Released
6,Nov 14 2022 11:34AM,250682,10,0086167288,ISDIN Corporation,Released
7,Nov 14 2022 11:34AM,250682,10,0086167292,ISDIN Corporation,Released
8,Nov 14 2022 11:34AM,250682,10,0086167293,ISDIN Corporation,Released
9,Nov 14 2022 11:34AM,250682,10,0086167294,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,250685,Released,1
25,250695,Released,1
26,250696,Released,1
27,250697,Executing,1
28,250697,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
250684,1.0,1.0
250685,NaN,1.0
250695,NaN,1.0
250696,NaN,1.0
250697,1.0,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
250634,0.0,2.0
250638,0.0,5.0
250659,0.0,6.0
250662,48.0,10.0
250664,15.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
250634,0,2
250638,0,5
250659,0,6
250662,48,10
250664,15,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,250634,0,2
1,250638,0,5
2,250659,0,6
3,250662,48,10
4,250664,15,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,250634,,2
1,250638,,5
2,250659,,6
3,250662,48,10
4,250664,15,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 14 2022 12:08PM,250697,19,"AdvaGen Pharma, Ltd"
3,Nov 14 2022 12:03PM,250696,12,Hush Hush
4,Nov 14 2022 12:01PM,250695,20,"Andelyn Biosciences, Inc."
5,Nov 14 2022 11:36AM,250685,10,"Senseonics, Incorporated"
6,Nov 14 2022 11:34AM,250682,10,ISDIN Corporation
70,Nov 14 2022 11:31AM,250684,10,Emerginnova
72,Nov 14 2022 11:22AM,250679,10,ISDIN Corporation
135,Nov 14 2022 11:15AM,250681,10,ISDIN Corporation
148,Nov 14 2022 11:07AM,250676,10,ISDIN Corporation
217,Nov 14 2022 11:05AM,250677,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Nov 14 2022 12:08PM,250697,19,"AdvaGen Pharma, Ltd",1,2
1,Nov 14 2022 12:03PM,250696,12,Hush Hush,,1
2,Nov 14 2022 12:01PM,250695,20,"Andelyn Biosciences, Inc.",,1
3,Nov 14 2022 11:36AM,250685,10,"Senseonics, Incorporated",,1
4,Nov 14 2022 11:34AM,250682,10,ISDIN Corporation,,64
5,Nov 14 2022 11:31AM,250684,10,Emerginnova,1,1
6,Nov 14 2022 11:22AM,250679,10,ISDIN Corporation,,63
7,Nov 14 2022 11:15AM,250681,10,ISDIN Corporation,,13
8,Nov 14 2022 11:07AM,250676,10,ISDIN Corporation,,69
9,Nov 14 2022 11:05AM,250677,10,ISDIN Corporation,,60


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 14 2022 12:08PM,250697,19,"AdvaGen Pharma, Ltd",2,1
1,Nov 14 2022 12:03PM,250696,12,Hush Hush,1,
2,Nov 14 2022 12:01PM,250695,20,"Andelyn Biosciences, Inc.",1,
3,Nov 14 2022 11:36AM,250685,10,"Senseonics, Incorporated",1,
4,Nov 14 2022 11:34AM,250682,10,ISDIN Corporation,64,
5,Nov 14 2022 11:31AM,250684,10,Emerginnova,1,1
6,Nov 14 2022 11:22AM,250679,10,ISDIN Corporation,63,
7,Nov 14 2022 11:15AM,250681,10,ISDIN Corporation,13,
8,Nov 14 2022 11:07AM,250676,10,ISDIN Corporation,69,
9,Nov 14 2022 11:05AM,250677,10,ISDIN Corporation,60,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 14 2022 12:08PM,250697,19,"AdvaGen Pharma, Ltd",2,1
1,Nov 14 2022 12:03PM,250696,12,Hush Hush,1,
2,Nov 14 2022 12:01PM,250695,20,"Andelyn Biosciences, Inc.",1,
3,Nov 14 2022 11:36AM,250685,10,"Senseonics, Incorporated",1,
4,Nov 14 2022 11:34AM,250682,10,ISDIN Corporation,64,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Nov 14 2022 12:08PM,250697,19,"AdvaGen Pharma, Ltd",2,1.0
1,Nov 14 2022 12:03PM,250696,12,Hush Hush,1,NaN
2,Nov 14 2022 12:01PM,250695,20,"Andelyn Biosciences, Inc.",1,NaN
3,Nov 14 2022 11:36AM,250685,10,"Senseonics, Incorporated",1,NaN
4,Nov 14 2022 11:34AM,250682,10,ISDIN Corporation,64,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 14 2022 12:08PM,250697,19,"AdvaGen Pharma, Ltd",2,1.0
1,Nov 14 2022 12:03PM,250696,12,Hush Hush,1,0.0
2,Nov 14 2022 12:01PM,250695,20,"Andelyn Biosciences, Inc.",1,0.0
3,Nov 14 2022 11:36AM,250685,10,"Senseonics, Incorporated",1,0.0
4,Nov 14 2022 11:34AM,250682,10,ISDIN Corporation,64,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3258691,397,51.0
12,250696,1,0.0
15,752007,10,17.0
19,501366,6,1.0
20,250695,1,0.0
21,501347,2,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3258691,397,51.0
1,12,250696,1,0.0
2,15,752007,10,17.0
3,19,501366,6,1.0
4,20,250695,1,0.0
5,21,501347,2,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,397,51.0
1,12,1,0.0
2,15,10,17.0
3,19,6,1.0
4,20,1,0.0
5,21,2,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,397.0
1,12,Released,1.0
2,15,Released,10.0
3,19,Released,6.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21
Status,,,,,,
Executing,51.0,0.0,17.0,1.0,0.0,0.0
Released,397.0,1.0,10.0,6.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21
0,Executing,51.0,0.0,17.0,1.0,0.0,0.0
1,Released,397.0,1.0,10.0,6.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21
0,Executing,51.0,0.0,17.0,1.0,0.0,0.0
1,Released,397.0,1.0,10.0,6.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()